Import modules.

In [12]:
import numpy as np
from qiskit import ClassicalRegister, QuantumRegister, QuantumCircuit
from qiskit_aer import Aer


Define the number of qubits available.

In [13]:
n_qubits = 29
dim_strings = n_qubits - 1

In [14]:
# To use local qasm simulator
backend = Aer.get_backend("aer_simulator")
shots = 1024

In [15]:
# Load saved credentials
# service = QiskitRuntimeService()
# options = Options()
# options.execution.shots = 1024

# # Set the backend
# backend = service.backend("ibmq_qasm_simulator")

# # Define the fidelity using Sampler(backend)
# fidelity = ComputeUncompute(sampler=Sampler(backend))

Define the mapping between letters and angles.

In [16]:
def string_to_angles(file_path):
    file = open(file_path, 'r')
    header_line = next(file)
    angles = []
    while 1:
        # read by character
        char = file.read(1)          
        if not char: 
            break
        if char == 'A':
            angles.append(np.pi/4)
        elif char == 'T':
            angles.append(3*np.pi/4)
        elif char == 'C':
            angles.append(5*np.pi/4)
        elif char == 'G':
            angles.append(7*np.pi/4)
    file.close()

    return angles

Create the quantum circuit for measuring the Hamming distance between strings.

In [18]:
def hamming_distance(theta, phi):
    dim_strings = len(theta)
    n_qubits = dim_strings + 1
    c = ClassicalRegister(1, 'c')
    q = QuantumRegister(n_qubits, 'q')
    circuit = QuantumCircuit(q, c)
    #########################################
    circuit.h(q[0])
    for i in range(1,n_qubits):
        circuit.h(q[i])
        if i % 2 == 0:
            circuit.rx(phi[i-2],q[i])
            circuit.rz(phi[i-1],q[i])
            circuit.cswap(q[0], q[i-1], q[i])
        elif i % 2 == 1:
            circuit.rx(theta[i-1],q[i])
            circuit.rz(theta[i],q[i])
    circuit.h(q[0])
    #########################################
    circuit.measure(q[0], c[0])
    circuit.reset(q)
    #########################################
    job = backend.run(circuit, shots=shots)
    result = job.result()   
    # plot_histogram(result.get_counts())
    #########################################
    return result

Load data and transform strings to lists of angles.

In [23]:
gene = string_to_angles('gene.fna')
patient_1 = string_to_angles('Patient_1_genome_fragment.fa')
patient_2 = string_to_angles('Patient_2_genome_fragment.fa')

Take sections of the gene to match the number of available qubits.

In [24]:
gene_sections = np.reshape(gene[:len(gene) - len(gene) % dim_strings],(-1, dim_strings))
gene_tail = gene[len(gene) - len(gene) % dim_strings:]

Call the function to compute the Hamming distance between the first section of the gene and same lenght sections of the genome.

In [ ]:
s = 0
i = 0
while i < len(patient_1)-dim_strings:
    result = hamming_distance(gene_sections[s], patient_1[i:i+dim_strings])
    probability = result.get_counts()['0']/shots
    if probability >= 0.95:
        if s < len(gene_sections) - 1:
            s += 1
            i = i + dim_strings
        elif s == len(gene_sections) - 1:
            result = hamming_distance(gene_tail, patient_1[i:i+len(gene_tail)])
            probability = result.get_counts()['0']/shots
            if probability >= 0.95:
                print('Gene found in genome of Patient #1')
                break
    else:
        s = 0
        i += 1
if i >= len(patient_1)-len(gene):
    print('Gene not found in genome of Patient #1')

Testing with short artificial strings.

In [ ]:
# gene = string_to_angles('gene_short.fna')
# gene_sections = np.reshape(gene[:len(gene) - len(gene) % dim_strings],(-1, dim_strings))
# gene_tail = gene[len(gene) - len(gene) % dim_strings:]
# test = string_to_angles('Test_genome_short.fa')

# s = 0
# i = 0
# while i < len(test)-dim_strings:
#     result = hamming_distance(gene_sections[s], test[i:i+dim_strings])
#     probability = result.get_counts()['0']/shots
#     if probability >= 0.95:
#         if s < len(gene_sections) - 1:
#             s += 1
#             i = i + dim_strings
#             print(s, '° part matching')
#         elif s == len(gene_sections) - 1:
#             result = hamming_distance(gene_tail, test[i:i+len(gene_tail)])
#             probability = result.get_counts()['0']/shots
#             if probability >= 0.95:
#                 print('Gene found in genome')
#                 break
#     else:
#         s = 0
#         i += 1

Demonstration circuit with parameters instead of data points.

In [ ]:
# n_qubits = 7
# dim_strings = n_qubits - 1

# theta = ParameterVector("θ", dim_strings)
# phi = ParameterVector("φ", dim_strings)

# c = ClassicalRegister(1, 'c')
# q = QuantumRegister(n_qubits, 'q')
# circuit = QuantumCircuit(q, c)

# circuit.h(q[0])
# for i in range(1,n_qubits):
#     circuit.h(q[i])
#     if i % 2 == 0:
#         circuit.rx(phi[i-2],q[i])
#         circuit.rz(phi[i-1],q[i])
#         circuit.cswap(q[0], q[i-1], q[i])
#     elif i % 2 == 1:
#         circuit.rx(theta[i-1],q[i])
#         circuit.rz(theta[i],q[i])
# circuit.h(q[0])
# circuit.measure(q[0], c[0])

# circuit.draw(output='mpl', style="iqp")